# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_csv = "cities.csv"
cities_data = pd.read_csv(cities_csv)
cities_data.head()



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Santa Cruz de la Sierra,-17.8000,-63.1667,18.03,68,75,12.86,BO,1660058961
1,Mataura,-46.1927,168.8643,-3.04,96,7,0.85,NZ,1660059349
2,Amapá,1.0000,-52.0000,33.03,39,67,2.47,BR,1660059350
3,Thompson,55.7435,-97.8558,14.09,82,75,6.17,CA,1660059351
4,Khatanga,71.9667,102.5000,14.03,76,77,4.94,RU,1660059353


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = cities_data[["Lat", "Lng"]]
humidity = cities_data["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Weather conditions: A max temperature lower than 27 degrees but higher than 21. Wind speed less than 4.5 m/s. Zero Clooudiness
ideal_weather = cities_data.loc[(cities_data["Max Temp"] < 27) & (cities_data["Max Temp"] > 21)
                               & (cities_data["Wind Speed"] < 4.5) & (cities_data["Cloudiness"] == 0),]
ideal_weather_clean = ideal_weather.dropna(axis="rows",how="any")
ideal_weather_clean

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
178,Nānākuli,21.3906,-158.1547,25.97,65,0,2.57,US,1660059610
195,Morondava,-20.2833,44.2833,23.70,77,0,4.43,MG,1660059635
273,Shīeli,44.1667,66.7500,24.71,27,0,4.08,KZ,1660059751
302,Kearney,40.6995,-99.0815,23.31,68,0,3.60,US,1660059799
323,Saint Anthony,45.0205,-93.2180,24.40,66,0,4.12,US,1660059834
416,Mölnlycke,57.6589,12.1179,21.08,66,0,4.12,SE,1660059985
478,Shakawe,-18.3667,21.8500,26.77,11,0,2.57,BW,1660060081
523,Senanga,-16.1167,23.2667,26.63,12,0,2.67,ZM,1660060164
527,Bismarck,46.8083,-100.7837,25.48,60,0,2.57,US,1660060169
528,Laguna,38.4210,-121.4238,21.63,74,0,3.09,US,1660060170


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather_clean
hotel_df["Hotel Name"] = " "
hotel_df = hotel_df.reset_index()

In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {"keyword": "hotel", "radius": 5000, "key":g_key}
for index, row in hotel_df.iterrows():
    try:
        lat = row["Lat"]
        lng = row["Lng"]
        params["location"] = f"{lat}, {lng}"
        response = requests.get(base_url, params=params).json()
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except:
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
      
        


In [8]:
hotel_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,178,Nānākuli,21.3906,-158.1547,25.97,65,0,2.57,US,1660059610,NA
1,195,Morondava,-20.2833,44.2833,23.70,77,0,4.43,MG,1660059635,Palissandre Cote Ouest resort & SPA
2,273,Shīeli,44.1667,66.7500,24.71,27,0,4.08,KZ,1660059751,NA
3,302,Kearney,40.6995,-99.0815,23.31,68,0,3.60,US,1660059799,Hampton Inn Kearney
4,323,Saint Anthony,45.0205,-93.2180,24.40,66,0,4.12,US,1660059834,Element Minneapolis Downtown
5,416,Mölnlycke,57.6589,12.1179,21.08,66,0,4.12,SE,1660059985,NA
6,478,Shakawe,-18.3667,21.8500,26.77,11,0,2.57,BW,1660060081,Ditlhapi lodge
7,523,Senanga,-16.1167,23.2667,26.63,12,0,2.67,ZM,1660060164,NA
8,527,Bismarck,46.8083,-100.7837,25.48,60,0,2.57,US,1660060169,Radisson Hotel Bismarck
9,528,Laguna,38.4210,-121.4238,21.63,74,0,3.09,US,1660060170,Hampton Inn & Suites Sacramento-Elk Grove Lagu...


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))